# Chapter 3

## 3.2.2 Create spark session and load json file

In [2]:
spark = SparkSession.builder \
    .appName('GitHub push counter') \
    .master('local') \
    .getOrCreate()

In [4]:
df_git_log = spark.read.json('dataIn/2015-03-01-0.json.gz')

In [25]:
df_git_log.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nullable = true)
 |    |    |    |    |-- href: strin

In [26]:
df_git_log.show(5)

+--------------------+--------------------+----------+--------------------+--------------------+------+--------------------+-----------------+
|               actor|          created_at|        id|                 org|             payload|public|                repo|             type|
+--------------------+--------------------+----------+--------------------+--------------------+------+--------------------+-----------------+
|[https://avatars....|2015-03-01T00:00:00Z|2614896652|[https://avatars....|[null,null,null,n...|  true|[23934080,Early-M...|      CreateEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896653|                null|[null,6dda286a3a1...|  true|[31481156,bezerra...|        PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896654|                null|[null,6089ce1d78d...|  true|[31475673,demianb...|        PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896656|                null|[created,null,[nu...|  true|[31481077,chrsmit...|IssueCommentEvent|

## 3.2.3 Filter only PushEvents
### filter takes a boolean condition or a SQL string 

In [11]:
df_push_type1 = df_git_log.filter(df_git_log.type == 'PushEvent')

In [18]:
df_push_type1.count()

8793

In [16]:
df_push_type1.show(5)

+--------------------+--------------------+----------+--------------------+--------------------+------+--------------------+---------+
|               actor|          created_at|        id|                 org|             payload|public|                repo|     type|
+--------------------+--------------------+----------+--------------------+--------------------+------+--------------------+---------+
|[https://avatars....|2015-03-01T00:00:00Z|2614896653|                null|[null,6dda286a3a1...|  true|[31481156,bezerra...|PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896654|                null|[null,6089ce1d78d...|  true|[31475673,demianb...|PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896663|                null|[null,3f55d3ea1fc...|  true|[31481269,ricardo...|PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896667|[https://avatars....|[null,15bf6c90255...|  true|[24902852,actorap...|PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896668| 

In [13]:
df_push_type2 = df_git_log.filter("type = 'PushEvent'")

In [19]:
df_push_type2.count()

8793

In [17]:
df_push_type2.show(5)

+--------------------+--------------------+----------+--------------------+--------------------+------+--------------------+---------+
|               actor|          created_at|        id|                 org|             payload|public|                repo|     type|
+--------------------+--------------------+----------+--------------------+--------------------+------+--------------------+---------+
|[https://avatars....|2015-03-01T00:00:00Z|2614896653|                null|[null,6dda286a3a1...|  true|[31481156,bezerra...|PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896654|                null|[null,6089ce1d78d...|  true|[31475673,demianb...|PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896663|                null|[null,3f55d3ea1fc...|  true|[31481269,ricardo...|PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896667|[https://avatars....|[null,15bf6c90255...|  true|[24902852,actorap...|PushEvent|
|[https://avatars....|2015-03-01T00:00:00Z|2614896668| 

## 3.2.4 GroupBy user and count

In [23]:
df_grouped = df_push_type1.groupBy('actor.login').count()

In [24]:
df_grouped.show(5)

+------------+-----+
|       login|count|
+------------+-----+
|john-griffin|    1|
|   digitized|    3|
| theCodeBear|    1|
|      WillHK|    1|
|  sakuya3834|    1|
+------------+-----+
only showing top 5 rows



### Sort/OrderBy

In [29]:
df_grouped.sort('count', ascending=False).show(5)

+------------------+-----+
|             login|count|
+------------------+-----+
|      greatfirebot|  192|
|diversify-exp-user|  146|
|     KenanSulayman|   72|
|        manuelrp07|   45|
|    mirror-updates|   42|
+------------------+-----+
only showing top 5 rows



In [31]:
df_grouped.orderBy('count', ascending=False).show(5)

+------------------+-----+
|             login|count|
+------------------+-----+
|      greatfirebot|  192|
|diversify-exp-user|  146|
|     KenanSulayman|   72|
|        manuelrp07|   45|
|    mirror-updates|   42|
+------------------+-----+
only showing top 5 rows



## 3.2.5 Excluding non employees